# definition

In [15]:
# Import the required modules
import os
import subprocess
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, ContentSettings
import re
import psycopg2


# Connect to your PostgreSQL database
conn = psycopg2.connect(
    dbname="NineHealth_2",
    user="postgres",
    password="1Lik39Health!",
    host="192.168.1.2",
    port="11433"
)
cursor = conn.cursor()

# Define the connection string and container name of the azure blob storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=9healthposedetectstorage;AccountKey=1hTQo55bdU4ciqPG9j/bMh4vD/XECkvnTL+D1eskL+KiiRknyb52Rw2mxaa93UBHMy6XuzWyr205+AStsHsT7g==;EndpointSuffix=core.windows.net"
container_name = "userviz"
content_settings = ContentSettings(content_type='video/mp4')
# Create a blob service client object
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Create a container client object
container_client = blob_service_client.get_container_client(container_name)

def convert(blob):
    blob_name = blob.name
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    # download_file_path = os.path.join(os.getcwd(), "in", re.sub(r'[\\/:*?"<>|]', '_', blob_name))
    download_file_path = os.path.join("D:\\2_userviz", blob_name.split("_")[0], re.sub(r'[\\/:*?"<>|]', '_', blob_name))
    with open(download_file_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())
    
    blob_name = convert_vietnamese_accent_to_english(blob_name).replace("(","-").replace(")","-")
    # Convert the downloaded file to the desired format using ffmpeg
    output_file_path = os.path.join(os.getcwd(), "out", re.sub(r'[\\/:*?"<>|]', '_', blob_name))
    subprocess.run(["ffmpeg", "-i", download_file_path, "-c:v", "libx264", "-c:a", "aac", "-strict", "experimental", "-b:a", "192k", "-movflags", "+faststart", output_file_path])
    return output_file_path, blob_name
    
def upload(blob_name, filepath): 
    # blob_name = blob.name # Add a suffix to the blob name to indicate the new format 
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name) 
    with open(filepath, "rb") as data: 
        # old_blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name) 
        # old_blob_client.delete_blob()
        blob_client.upload_blob(data, content_settings=content_settings, overwrite=True)
        
def update_database_url(user_id, start_time, video_url,set_name):
    # Extract the video name from the video_url
    # name = re.search(r'([^/]+)\.mp4$', video_url).group(1)
    formatted_start_time = start_time.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    
    # SQL UPDATE statement to update the video_url in the database
    update_query = """
    UPDATE users_connect
    SET video_url = %s
    WHERE user_id = %s
    AND start_time::text LIKE %s
    AND "name" LIKE %s
    """
    # Execute the update query
    cursor.execute(update_query, (video_url, user_id, formatted_start_time, set_name))
    conn.commit()
    print("Database updated successfully")
    
def convert_vietnamese_accent_to_english(text):
    """
    Convert Vietnamese accents to English
    """
    vietnamese_accents = {
        'à': 'a', 'á': 'a', 'ả': 'a', 'ã': 'a', 'ạ': 'a',
        'ă': 'a', 'ằ': 'a', 'ắ': 'a', 'ẳ': 'a', 'ẵ': 'a', 'ặ': 'a',
        'â': 'a', 'ầ': 'a', 'ấ': 'a', 'ẩ': 'a', 'ẫ': 'a', 'ậ': 'a',
        'đ': 'd',
        'è': 'e', 'é': 'e', 'ẻ': 'e', 'ẽ': 'e', 'ẹ': 'e',
        'ê': 'e', 'ề': 'e', 'ế': 'e', 'ể': 'e', 'ễ': 'e', 'ệ': 'e',
        'ì': 'i', 'í': 'i', 'ỉ': 'i', 'ĩ': 'i', 'ị': 'i',
        'ò': 'o', 'ó': 'o', 'ỏ': 'o', 'õ': 'o', 'ọ': 'o',
        'ô': 'o', 'ồ': 'o', 'ố': 'o', 'ổ': 'o', 'ỗ': 'o', 'ộ': 'o',
        'ơ': 'o', 'ờ': 'o', 'ớ': 'o', 'ở': 'o', 'ỡ': 'o', 'ợ': 'o',
        'ù': 'u', 'ú': 'u', 'ủ': 'u', 'ũ': 'u', 'ụ': 'u',
        'ư': 'u', 'ừ': 'u', 'ứ': 'u', 'ử': 'u', 'ữ': 'u', 'ự': 'u',
        'ỳ': 'y', 'ý': 'y', 'ỷ': 'y', 'ỹ': 'y', 'ỵ': 'y'
    }
    for k, v in vietnamese_accents.items():
        text = text.replace(k, v)
    return text

In [16]:
from datetime import datetime

# Dictionary mapping user_id to blob name prefix
user_blob_prefix = {
    176: 'minhlong',
    # 133: 'vuvietquang',
    # 144: 'paulpzo12'
}
# Loop through the user_id and blob name dictionary
for user_id, blob_prefix in user_blob_prefix.items():
    start_time_query = '2023-12-31'  # Modify this accordingly
    start_time_condition = datetime.strptime(start_time_query, '%Y-%m-%d').strftime('%d-%m-%Y')
    blob_name_condition = f"{blob_prefix}_{start_time_query}"  # Modify this accordingly
    blob_list = container_client.list_blobs(blob_name_condition)
    
    select_query = """
        SELECT user_connect_id, user_id, "name", start_time
        FROM users_connect
        WHERE user_id = %s
        AND start_time::text LIKE %s
        ORDER BY start_time ASC
    
    """

    cursor.execute(select_query, (user_id, f'%{start_time_query}%'))
    records = cursor.fetchall()

    # Loop through retrieved records and update Azure Blob Storage URLs
    for record in records:
        user_connect_id, user_id, set_name, start_time = record
        # Construct the video URL based on your naming convention
        video_name = f"{set_name.replace(' ', '_').lower().replace('(', '').replace(')', '')}"  # Modify this naming convention if needed
        # Update the URL in Azure Blob Storage
        blob_list = container_client.list_blobs(blob_name_condition)
        for blob in blob_list:
            blob_name = blob.name
            if video_name not in blob_name: continue
            if f"{blob_prefix}_{start_time_query}" not in blob_name: continue
                
            # re_blob_name = convert_vietnamese_accent_to_english(re.sub(r'[\\/:*?"<>|]', '_', blob_name))
            # re_blob_name = re_blob_name.replace("(", "-").replace(")", "-")

            # output_file_path = os.path.join(os.getcwd(), "out", re_blob_name)
            print("Processing:", blob_name)
            output_file_path, blob_name_no_accents = convert(blob)
            upload(blob_name_no_accents, output_file_path)
            
            # Constructing the blob URL
            blob_url = f"https://{container_client.account_name}.blob.core.windows.net/{container_client.container_name}/{blob_name_no_accents}"
            
            os.remove(output_file_path)
            print("Done:", blob_name)
            
            update_database_url(user_id, start_time, blob_url, set_name)

cursor.close()
conn.close()

In [ ]:
blob_list = container_client.list_blobs()
for blob in blob_list:
    blob_client = container_client.get_blob_client(blob)
    print(blob_client.url)
    print("Done:", blob_name)
            

In [ ]:
!powershell C:\Users\9Health\Documents\0_9HA\9h-pipelines\contents\move-mp4.ps1
